# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
H. Baehr  ->  H. Baehr  |  ['H. Baehr']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
F. Xu  ->  F. Xu  |  ['F. Xu']


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 100 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.00235


extracting tarball to tmp_2511.00235... done.
Retrieving document from  https://arxiv.org/e-print/2511.00513


extracting tarball to tmp_2511.00513...

 done.


...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.01750


extracting tarball to tmp_2511.01750...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


Found 145 bibliographic references in tmp_2511.01750/MRybak_PRUSSIC_III.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.01167-b31b1b.svg)](https://arxiv.org/abs/2511.01167) | **Discovery of Seven Cold and Distant Brown Dwarfs with JWST RUBIES**  |
|| S. J. Morrissey, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-11-04*|
|*Comments*| *41 pages, 7 figures (AAS manuscript format); submitted to AJ*|
|**Abstract**|            We report near-infrared spectral model fits to seven distant L- and T-type dwarfs observed with the JWST Near Infrared Spectrograph (NIRSpec) as part of the Red Unknowns: Bright Infrared Extragalactic Survey (RUBIES). Comparison of 0.9-2.5 $\mu$m near-infrared spectra of these sources to spectral standards indicates spectral types spanning L1 to T8, and spectrophotometric distances spanning 800-3,000 pc. Fits to three grids of spectral models yield atmosphere parameters and spectrophotometric distances largely consistent with our classifications, although fits to L dwarf spectra indicate missing components to the models. Three of our sources have vertical displacements from the Galactic plane exceeding 1 kpc, and have high probabilities of membership in the Galactic thick disk population. Of these, the L dwarf RUBIES-BD-3 (RUBIES-EGS-3081) is well-matched to subdwarf standards, while the early T dwarf RUBIES-BD-5 (RUBIES-UDS-170428) is best fit by metal-poor atmosphere models; both many be a thick disk or halo brown dwarfs. We critically examine the 1-5 $\mu$m spectra of the current sample of 1-2 kpc mid- and late-T dwarfs, finding that temperature, surface gravity, metallicity, and vertical mixing efficiency can all contribute to observed variations in near-infrared spectral structure and the strength of the 4.2 $\mu$m CO band. This work aims to guide ongoing JWST, Euclid, and other space-based spectral surveys that are expected to uncover thousands of low-temperature stars and brown dwarfs throughout the Milky Way.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.01285-b31b1b.svg)](https://arxiv.org/abs/2511.01285) | **The ALMA-QUARKS survey: Hot Molecular Cores are a long-standing phenomenon in the evolution of massive protostars**  |
|| D. Meng, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-11-04*|
|*Comments*| *resubmitted to ApJ after taking into account referee's comments*|
|**Abstract**|            We present an analysis of the QUARKS survey sample, focusing on protoclusters where Hot Molecular Cores (HMCs, traced by CH3CN(12--11)) and UC HII regions (traced by H30\alpha/H40\alpha) coexist. Using the high-resolution, high-sensitivity 1.3 mm data from the QUARKS survey, we identify 125 Hot Molecular Fragments (HMFs), which represent the substructures of HMCs at higher resolution. From line integrated intensity maps of CH3CN(12--11) and H30\alpha, we resolve the spatial distribution of HMFs and UC HII regions. By combining with observations of CO outflows and 1.3 mm continuum, we classify HMFs into four types: HMFs associated with jet-like outflow, with wide-angle outflow, with non-detectable outflow, and shell-like HMFs near UC HII regions. This diversity possibly indicates that the hot core could be polymorphic and long-standing phenomenon in the evolution of massive protostars. The separation between HMFs and H30\alpha/H40\alpha emission suggests that sequential high-mass star formation within young protoclusters is not likely related to feedback mechanisms.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.01750-b31b1b.svg)](https://arxiv.org/abs/2511.01750) | **PRUSSIC III - ALMA and NOEMA survey of dense gas in high-redshift star-forming galaxies**  |
|| M. Rybak, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-11-04*|
|*Comments*| *Submitted to A&A. First version 14th August 2025; revised 4th November 2025*|
|**Abstract**|            Characterising the relationship between dense gas and star formation is critical for understanding the assembly of galaxies throughout cosmic history. However, due to the faintness of standard dense-gas tracers - HCN, HCO+, and HNC - dense gas in high-redshift galaxies remains largely unexplored. We present ALMA and NOEMA observations targeting HCN/HCO+/HNC (3-2) and (4-3) emission lines in eleven (mostly) gravitationally lensed dusty star-forming galaxies (DSFGs) at redshift z = 1.6--3.2. We detect at least one line in 10 out of 11 galaxies. Altogether, we detect 34 dense-gas transitions, more than quadrupling the number of extant high-redshift detections. Additionally, in two targets, we detect lower-abundance CO isotopologues 13^CO and C^18O, as well as CN emission. We derive excitation coefficients for HCN, HCO+ and HNC in DSFGs, finding them to be systematically higher than those in nearby luminous infrared galaxies. Assuming a canonical dense-mass conversion factor (alpha_HCN = 10), we find that DSFGs have shorter dense- gas depletion times (median 23 Myr) than nearby galaxies (~60 Myr), with a star-forming efficiency per free-fall time of 1-2%, a factor of a few higher than in local galaxies. We find a wide range of dense-gas fractions, with HCN/CO ratios ranging between 0.01 and 0.15. Finally, we put the first constraints on the redshift evolution of the cosmic dense-gas density, which increases by a factor of 7+/-4 between z = 0 and z = 2.5, consistent with the evolution of the cosmic molecular-gas density.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.01717-b31b1b.svg)](https://arxiv.org/abs/2511.01717) | **Disk Evolution Study Through Imaging of Nearby Young Stars (DESTINYS): V721 CrA and BN CrA have wide and structured disks in polarised IR**  |
|| G. Columba, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-11-04*|
|*Comments*| *Accepted for pubblication on Astronomy & Astrophysics (A&A). 10 (+4) pages, 7 (+5) figures (appendix)*|
|**Abstract**|            We present near-infrared scattered-light observations of the disks around two stars of the Corona Australis star-forming region, V721 CrA, and BN CrA, obtained with VLT/SPHERE, in the H band, as part of the DESTINYS large programme. Our objective is to analyse the morphology of these disks, and highlight their main properties. We adopt an analytical axisymmetric disk model to fit the observations and perform a regression on key disk parameters, namely the dust mass, the height profile, and the inclination. We use RADMC-3D code to produce synthetic observations of the analytical models, with full polarised scattering treatment. Both stars show resolved and extended disks with substructures in the near-IR. The disk of V721 CrA is vertically thicker, radially smaller (120 au), and brighter than BN CrA (190 au). It also shows spiral arms in the inner regions. The disk of BN CrA shows a dark circular lane, which could be either an intrinsic dust gap or a self-cast shadow, and a brightness enhancement along the disk minor axis. Both disks are compatible with the evolutionary stage of their parent subgroup within the CrA region: V721 CrA belongs to the "on-cloud" part of CrA, which is dustier, denser and younger, whereas BN CrA is found on the outskirts of the older "off-cloud" group.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.01733-b31b1b.svg)](https://arxiv.org/abs/2511.01733) | **A Possible "Too-Many-Satellites" Problem in the Isolated Dwarf Galaxy DDO 161**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-11-04*|
|*Comments*| *9 pages, submitted to ApJL*|
|**Abstract**|            The abundance of satellite galaxies provides a direct test of $\Lambda$CDM on small scales. While satellites of Milky Way-mass galaxies are well studied, those of dwarf galaxies remain largely unexplored. We present a systematic search for satellites around the isolated dwarf galaxy DDO 161 ($M_\star \approx 10^{8.4}\, M_\odot$) at a distance of 6 Mpc. We identify eight satellite candidates within the projected virial radius and confirm four satellites through surface brightness fluctuation distance measurements from deep Magellan imaging data. With four confirmed satellites above $M_{\star}^{\rm sat} > 10^{5.4}\, M_\odot$, DDO 161 is the most satellite-rich dwarf galaxy known to date. We compare this system with predictions from the TNG50 cosmological simulation, combined with currently established galaxy-halo connection models calibrated on Milky Way satellites, and find that DDO 161 has a satellite abundance far exceeding all current expectations. The rich satellite system of DDO 161 offers new insight into how low-mass galaxies occupy dark matter halos in low-density environments and may provide new constraints on the nature of dark matter.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.00235-b31b1b.svg)](https://arxiv.org/abs/2511.00235) | **Chemodynamic evidence of pristine gas accretion in the void galaxy VGS 12**  |
|| E. Egorova, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-11-04*|
|*Comments*| *15 pages, 11 figures, 4 tables. Accepted for publication in A&A*|
|**Abstract**|            Accretion of metal-poor gas is expected to be an important channel of gas replenishment in galaxy evolution studies. However, observational evidence of this process is still relatively scarce. The unusual polar disk galaxy VGS 12 was found in the Void Galaxy Survey. It appears to be isolated and resides in the cosmological wall between two large voids. The suggested formation scenario for this peculiar system is accretion of metal-poor gas from the void interior. We present the data on the gas-phase chemical abundance of VGS 12 obtained with the Russian 6m telescope BTA. We complement our analysis with HI data obtained with VLA and the data on the kinematics of the ionized gas. VGS 12 appears to be a strong outlier from the "metallicity - luminosity" relation, with gas oxygen abundance ~0.7dex lower than expected for its luminosity. The nitrogen abundance, on the other hand, is higher than what is typically observed in galaxies with similar metallicity, but is consistent with the metallicity expected given its luminosity. Such behavior is what is expected in the case of metal-poor gas accretion. The HI reveals clear morphological and kinematical asymmetry between the northern and southern parts of the disk, which are likely related to its unsettled state due to the recent accretion event. The kinematics of the ionized gas seen in Halpha reveal prolate rotation and follow closely the rotation of the HI disk, so we suggest this is accreted HI gas ionized by the stars in the central region of the galaxy. Together, our findings provide strong, multiwavelength evidence of ongoing cold gas accretion in a galaxy caught in the act of growing from the cosmic web. This is one of the very few individual galaxies where a convincing case can be made for such a process, and demonstrates the potential for cold accretion to contribute to galaxy growth even in the low-redshift universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.00513-b31b1b.svg)](https://arxiv.org/abs/2511.00513) | **On the Gravitational Collapse of Small Dust Grains in Self-gravitating Disk Structures**  |
|| <mark>H. Baehr</mark>, K. Rice, C.-C. Yang, C. Hall |
|*Appeared on*| *2025-11-04*|
|*Comments*| *15 pages, 8 figures, accepted for publication in ApJ*|
|**Abstract**|            Planet formation may begin much earlier than previously expected, when the protoplanetary disk is still massive and gravitationally unstable. It has been proposed that solid grains can concentrate in the spiral arms of self-gravitating disks, leading to the formation of planetary embryos or cores that can greatly accelerate the process of planet formation. We perform hydrodynamic simulations of self-gravitating gas and even smaller dust grains than previously investigated in 3-dimensional shearing box simulations to explore the conditions necessary to form these planetary seeds. Focusing on small grains of dimensionless stopping time $\mathrm{St}=0.01$ and shorter, we find that disk metallicities $Z \gtrsim 0.02$ can overcome the disruptive effects of dust diffusion among these small dust grains. In the outer reaches of a gravitationally unstable disk, these models correspond to grains of approximately 1$\,mm$ and lead to planetary embryos between 0.1 and 1 Earth mass. The formation of these planetary embryos could therefore reduce the time needed for planet assembly, particularly in the outer regions of the disk where coagulation timescales are longer and solid growth is limited.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \c at position 14</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.01167/./mcmcfit_BD-1_dback24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-1_sand24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-2_elfowl24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-5_elfowl24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-5_dback24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-1_elfowl24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-2_dback24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-2_sand24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-3_elfowl24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-3_dback24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-3_sand24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-4_elfowl24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-4_dback24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-4_sand24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./mcmcfit_BD-5_sand24_JWST-NIRSPEC-PRISM.png', 'tmp_2511.01167/./rubbd3_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\logg}{\log{\mathrm{g}}}$
$\newcommand{\kzz}{\log\kappa_\mathrm{zz}}$
$\newcommand{\fsed}{f_{sed}}$
$\newcommand{\rjup}{R_\mathrm{Jup}}$
$\newcommand{\meth}{CH_4}$
$\newcommand{\wat}{H_2O}$
$\newcommand{\chhhh}{CH_4}$
$\newcommand{\coo}{CO_2}$
$\newcommand{\phhh}{PH_3}$
$\newcommand{\nhhh}{NH_3}$
$\newcommand{\hho}{H_2O}$
$\newcommand{\hh}{H_2}$
$\newcommand{\chisquare}{\chi^2}$
$\newcommand{\ldl}{\lambda/\Delta\lambda}$
$\newcommand{\adamb}[1]{\par{\bfseries \color{purple}AB: #1 \par}}$
$\newcommand{\micron}{\mathrm{\mu}m}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>



<div id="title">

# Discovery of Seven Cold and Distant Brown Dwarfs with JWST RUBIES

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.01167-b31b1b.svg)](https://arxiv.org/abs/2511.01167)<mark>Appeared on: 2025-11-04</mark> -  _41 pages, 7 figures (AAS manuscript format); submitted to AJ_

</div>
<div id="authors">

S. J. Morrissey, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We report near-infrared spectral model fits to seven distant L- and T-type dwarfs observed with the JWST Near Infrared Spectrograph (NIRSpec) as part of the Red Unknowns: Bright Infrared Extragalactic Survey (RUBIES).Comparison of 0.9--2.5 ${\micron}$ near-infrared spectra of these sources to spectral standards indicates spectral types spanning L1 to T8, and spectrophotometric distances spanning 800--3,000 pc.Fits to three grids of spectral models yield atmosphere parameters and spectrophotometric distances largelyconsistent with our classifications, although fits to L dwarf spectra indicate missing components to the models.Three of our sources have vertical displacements from the Galactic plane exceeding 1 kpc, and have high probabilities of membership in the Galactic thick disk population.Of these, the L dwarfRUBIES-BD-3 (RUBIES-EGS-3081) is well-matched to subdwarf standards,while the early T dwarf RUBIES-BD-5 (RUBIES-UDS-170428) is best fit by metal-poor atmosphere models; both many be a thick disk or halo brown dwarfs.We critically examine the 1--5 ${\micron}$ spectra of the current sample of 1--2 kpc mid- and late-T dwarfs, finding that temperature, surface gravity, metallicity, and vertical mixing efficiency can all contribute to observed variations in near-infrared spectral structure and the strength of the 4.2 ${\micron}$ ${\coo}$ band.This work aims to guide ongoing JWST, Euclid, and other space-based spectral surveys that are expected to uncover thousands of low-temperature stars and brown dwarfs throughout the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2511.01167/./mcmcfit_BD-1_dback24_JWST-NIRSPEC-PRISM.png" alt="Fig6.1" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-1_sand24_JWST-NIRSPEC-PRISM.png" alt="Fig6.2" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_elfowl24_JWST-NIRSPEC-PRISM.png" alt="Fig6.3" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_elfowl24_JWST-NIRSPEC-PRISM.png" alt="Fig6.4" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_dback24_JWST-NIRSPEC-PRISM.png" alt="Fig6.5" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-1_elfowl24_JWST-NIRSPEC-PRISM.png" alt="Fig6.6" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_dback24_JWST-NIRSPEC-PRISM.png" alt="Fig6.7" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_sand24_JWST-NIRSPEC-PRISM.png" alt="Fig6.8" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_elfowl24_JWST-NIRSPEC-PRISM.png" alt="Fig6.9" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_dback24_JWST-NIRSPEC-PRISM.png" alt="Fig6.10" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_sand24_JWST-NIRSPEC-PRISM.png" alt="Fig6.11" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_elfowl24_JWST-NIRSPEC-PRISM.png" alt="Fig6.12" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_dback24_JWST-NIRSPEC-PRISM.png" alt="Fig6.13" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_sand24_JWST-NIRSPEC-PRISM.png" alt="Fig6.14" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_sand24_JWST-NIRSPEC-PRISM.png" alt="Fig6.15" width="6%"/>

**Figure 6. -** Same as Figure \ref{fig:nir} for the full NIRSpec/Prism 0.9--5.1 {$\micron$} spectral range. (*fig:nirspec*)

</div>
<div id="div_fig2">

<img src="tmp_2511.01167/./rubbd3_dwarfcomp.png" alt="Fig3.1" width="25%"/><img src="tmp_2511.01167/./rubbd3_dsdcomp.png" alt="Fig3.2" width="25%"/><img src="tmp_2511.01167/./rubbd3_sdcomp.png" alt="Fig3.3" width="25%"/><img src="tmp_2511.01167/./rubbd3_esdcomp.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** Comparison of the 0.9--2.5 {$\micron$} NIRSpec/Prism spectrum of RUBIES-BD-3 (black line) to
    L6 to T2 dwarf standards (left panel);
    d/sdL6 to d/sdT2 mild subdwarf standards (center panel);
    sdL5, sdT0, sdT1, and sdT3 subdwarf standards (top right panel); and
    esdL6, esdL8, and esdT3 extreme subdwarf standards (bottom right panel).
    Standards are color-coded by spectral type.
    All spectra are normalized and offset by constants to facilitate comparison. (*fig:rubbd3_classification*)

</div>
<div id="div_fig3">

<img src="tmp_2511.01167/./mcmcfit_BD-1_dback24_NIR.png" alt="Fig4.1" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-1_sand24_NIR.png" alt="Fig4.2" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_elfowl24_NIR.png" alt="Fig4.3" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_sand24_NIR.png" alt="Fig4.4" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_dback24_NIR.png" alt="Fig4.5" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_sand24_NIR.png" alt="Fig4.6" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_elfowl24_NIR.png" alt="Fig4.7" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_dback24_NIR.png" alt="Fig4.8" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-1_elfowl24_NIR.png" alt="Fig4.9" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-2_dback24_NIR.png" alt="Fig4.10" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-3_elfowl24_NIR.png" alt="Fig4.11" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_elfowl24_NIR.png" alt="Fig4.12" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_dback24_NIR.png" alt="Fig4.13" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-4_sand24_NIR.png" alt="Fig4.14" width="6%"/><img src="tmp_2511.01167/./mcmcfit_BD-5_sand24_NIR.png" alt="Fig4.15" width="6%"/>

**Figure 4. -** MCMC fits to NIRSpec/Prism data constrained to the NIR (0.9-2.4 {$\micron$}) range.
    Each row corresponds to one of the RUBIES brown dwarf spectra scaled to apparent F444W magnitudes (black lines), while the columns separate fits to the Elf Owl (left), Diamondback (middle) and SAND (right) models. Both the best-fit models (solid magenta lines) and draws from the posterior distribution (semi-transparent magenta lines) are shown, and the $\pm$1$\sigma$ spectral uncertainties are indicated by the grey band.
    Below each spectral comparison, we compare the difference between the source and best-fit model fluxes (O-C as black lines) to the $\pm$1$\sigma$ spectral uncertainties (grey bands).
    Panels that are surrounded by boxes indicate the models that provide the best fits, including equivalent cases (see Table \ref{tab:summary}). (*fig:nir*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.01167"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\radex}{\texttt{SpectralRadex}}$
$\newcommand{\astrodendro}{\texttt{astrodendro}}$
$\newcommand{\spectuner}{\texttt{spectuner}}$
$\newcommand{\HII}{HC/UC H {\sc ii}}$
$\newcommand{\CO}{^{12}CO (2--1)}$
$\newcommand{\kms}{km s\rm ^{-1}}$
$\newcommand{\Kkms}{K km s\rm ^{-1}}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\rms}{\sigma}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{0.8}$
$\newcommand{\arraystretch}{0.8}$
$\newcommand{\arraystretch}{0.8}$
$\newcommand\ch{3cn}$
$\newcommand{\1}{2CO}$</div>



<div id="title">

# The ALMA-QUARKS survey: Hot Molecular Cores are a long-standing phenomenon in the evolution of massive protostars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.01285-b31b1b.svg)](https://arxiv.org/abs/2511.01285)<mark>Appeared on: 2025-11-04</mark> -  _resubmitted to ApJ after taking into account referee's comments_

</div>
<div id="authors">

D. Meng, et al. -- incl., <mark>F. Xu</mark>

</div>
<div id="abstract">

**Abstract:** We present an analysis of the QUARKS survey sample, focusing on protoclusters where Hot Molecular Cores (HMCs, traced by $\ch$ 3cn (12--11)) and $\HII$ regions (traced by H30 $\alpha$ /H40 $\alpha$ ) coexist. \textcolor{black}{Using the high-resolution, high-sensitivity 1.3 mm data from the QUARKS survey, we identify 125 Hot Molecular Fragments (HMFs), which represent the substructures of HMCs at higher resolution.} \textcolor{black}{From} line integrated intensity maps of \textcolor{black}{\ch3cn ($12_3$--$11_3$)} and H30 $\alpha$ , we resolve the spatial distribution of HMFs and $\HII$ regions. By combining with observations of $\CO$ outflows and 1.3 mm continuum, we classify \textcolor{black}{HMFs into four types: HMFs associated with jet-like outflow, with wide-angle outflow, with non-detectable outflow, and shell-like HMFs near \HII regions} .This diversity possibly indicates that the hot core could be polymorphic and long-standing phenomenon in the evolution of massive protostars.The separation between \textcolor{black}{HMFs} and H30 $\alpha$ /H40 $\alpha$ emission suggests that sequential high-mass star formation within young protoclusters is not likely related to feedback mechanisms.

</div>

<div id="div_fig1">

<img src="tmp_2511.01285/./example.png" alt="Fig2" width="100%"/>

**Figure 2. -** Images of the hydrogen  recombination lines emssion and the outflow for four exemplar sources. The background shows the three-color image composed by the red-shifted $\CO$(red), H30$\alpha$ integrated line emission (green) and the blue-shifted $\CO$(blue). The gray contours and orange contours represent the 1.3 mm continuum and the \ch3cn ($12_3$--$11_3$) integrated line emission, respectively.
    H30$\alpha$ integrated velocity range is [Vlsr-40 $\kms$, Vlsr+40 $\kms$], where Vlsr is the central velocity (Table \ref{tab:hii} column (9)). For \ch3cn ($12_3$--$11_3$), the integrated velocity ranges are different for different fields.
    The contour levels were plotted from 3 $\rms$ to the peak intensity of the field, with 8 logarithmically spaced contours between these values.
    White markers mark the positions of different type of \textcolor{black}{HMFs}: triangle (jet-like outflow), pentagon (wide-angle outflow), star (no/weak outflow) and cross (shell-like shape).
    The synthesized beams are shown in the lower left corner, and the scale bar is indicated in the lower right corner of each image.
     (*fig:example*)

</div>
<div id="div_fig2">

<img src="tmp_2511.01285/./multiscale.png" alt="Fig3" width="100%"/>

**Figure 3. -** Example of $\astrodendro$ results at different scales. The left, middle, and right panels show data from ACA  (quarks2) , ACA+TM2  ([Yang, Liu and Liu 2025]()) , and ACA+TM1+TM2  ([Liu, Liu and Zhu 2024]()) , respectively. The background shows the 1.3 mm continuum. The red contours represent the integrated intensity of \ch3cn (12–11), with contour levels ranging from 3 $\rms$ to 0.95 $\times$ the peak value, evenly spaced into five levels. The green ellipses outline the structures identified by $\astrodendro$ . The synthesized beams (left panel: $\sim$5$\arcsec$; middle panel: $\sim$ 1$\arcsec$; right panel: $\sim$ 0.3$\arcsec$) are shown in the lower left corner, and the scale bar is indicated in the lower right corner of each panel.
     (*fig:multiscale*)

</div>
<div id="div_fig3">

<img src="tmp_2511.01285/./separa.png" alt="Fig7.1" width="50%"/><img src="tmp_2511.01285/./hotcoresize.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Left column: histogram (bottom) and cumulative density distribution (top) for the separation between different kinds of \textcolor{black}{HMFs} and the nearest $\HII$ regions.
    For the shell-like type, the nearest $\HII$ region is defined as the $\HII$ region enclosed by the \textcolor{black}{nearby}\ch3cn emission, whereas for the other types, the nearest $\HII$ region is taken to be the closest $\HII$ region within the entire image. (Figure \ref{fig:hotcores_outflows}).
    The red and green vertical dashed lines indicate the median value of externally (shell-like) and internally \textcolor{black}{candidate}(with and without outflow) heated \textcolor{black}{HMFs}, respectively.
    The separation is calculated using the distance from [Liu, Liu and Zhu (2024)]() and the intensity peak coordinates of Table \ref{tab:hotcores}, \ref{tab:hii}. Right column: histogram (bottom) and cumulative density distribution (top) for the FWHM ($=\sqrt{\mathrm{Maj\_FWHM}\times \mathrm{Min\_FWHM}}\times \mathrm{distance}$) of different kinds of \textcolor{black}{HMFs}.
    The red and blue vertical dashed lines indicate the corresponding median value.
     (*fig:hist*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.01285"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# PRUSSIC III -  ALMA and NOEMA survey of dense gas in high-redshift star-forming galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.01750-b31b1b.svg)](https://arxiv.org/abs/2511.01750)<mark>Appeared on: 2025-11-04</mark> -  _Submitted to A&A. First version 14th August 2025; revised 4th November 2025_

</div>
<div id="authors">

M. Rybak, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Characterising the relationship between dense gas and star formation is critical for understanding the assembly of galaxies throughout cosmic history. However, due to the faintness of standard dense-gas tracers -- HCN, HCO $^+$ , and HNC -- dense gas in high-redshift galaxies remainslargely unexplored.  We present ALMA and NOEMA observations targeting HCN/HCO $^+$ /HNC (3--2) and (4--3) emission lines in eleven (mostly) gravitationally lensed dusty star-forming galaxies (DSFGs) at redshift $z=1.6-3.2$ .  We detect at least one line in 10 out of 11 galaxies. Altogether, we detect 34 dense-gas transitions, more than quadrupling the number of extant high-redshift detections. Additionally, in two targets, we detect lower-abundance CO isotopologues $^{13}$ CO and C $^{18}$ O, as well as CN emission.  We derive excitation coefficients for HCN, HCO $^+$ and HNC in DSFGs, finding them to be systematically higher than those in nearby luminous infrared galaxies.  Assuming a canonical dense-mass conversion factor ( $\alpha_\mathrm{HCN}=10$ ), we find that DSFGs have shorter dense-gas depletion times (median 23 Myr) than nearby galaxies ( $\approx60$ Myr), with a star-forming efficiency per free-fall time of 1--2 \% , a factor of a few higher than in local galaxies. We find a wide range of dense-gas fractions, with HCN/CO ratios ranging between 0.01 and 0.15. Finally, we put the first constraints on the redshift evolution of the cosmic dense-gas density, which increases by a factor of $7\pm4$ between $z=0$ and $z=2.5$ , consistent with the evolution of the cosmic molecular-gas density.

</div>

<div id="div_fig1">

<img src="tmp_2511.01750/./cleaned_SDP9_HCN43_1fwhm.statwt.contsub.png" alt="Fig8.1" width="33%"/><img src="tmp_2511.01750/./cleaned_SDP9_HCO+43_1fwhm.statwt.contsub.png" alt="Fig8.2" width="33%"/><img src="tmp_2511.01750/./cleaned_SDP9_HNC43_1fwhm.statwt.contsub.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** NOEMA narrow-band images of the HCN/HCO$^+$/HNC lines for SDP.9, which is detected in all three transitions. For the remaining sources, see Appendix. Contours start at $\pm$2$\sigma$ and increase in steps of 2$\sigma$.  (*fig:noema_hcnhcohnc*)

</div>
<div id="div_fig2">

<img src="tmp_2511.01750/./fig_SFE_20250717.png" alt="Fig11.1" width="50%"/><img src="tmp_2511.01750/./fig_FIR_HCN_over_CO_2025.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** {Left:} Ratio of HCN(1--0) and FIR luminosities (a proxy for star-forming efficiency) as a function of star-formation rate. The dotted lines indicate the mean and 1$\sigma$ scatter for local galaxies from the EMPIRE sample  ([Jiménez-Donaire, Bigiel and Leroy 2019]()) . We infer HCN(1--0) luminosities for our DSFGs using the following excitation coefficients: $r_\mathrm{31}=0.41$  and $r_\mathrm{41}=0.59$. {Right:} Ratio of  HCN(1--0) and CO(1--0) luminosities (a proxy for the dense-gas fraction). The inferred HCN/CO ratios span $\approx$1 dex, suggesting that dense-gas fraction in high-z DSFGs vary significantly source-to-source.  (*fig:SFE*)

</div>
<div id="div_fig3">

<img src="tmp_2511.01750/./fig_ID_FWHM_CO_HCN_HCO+_HNC_sorted.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of CO, HCN, HCO$^+$ and HNC linewidths in individual galaxies, ordered by increasing CO linewidth. The linewidths of different tracers are generally consistent within 1$\sigma$ uncertainties. Note the large discrepancies between HCN/HCO$^+$ and HNC linewidths in G04v1.40 and J1202. (*fig:fwhm_fwhm*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.01750"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

117  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
